In [1]:
# 필요한 라이브러리 및 모듈 임포트
import os
import boto3
import sagemaker
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Load data
business_df = pd.read_csv('./data/restaurant_unique_business.csv')
user_df = pd.read_csv('./data/restaurant_unique_user.csv')
rest_df = pd.read_csv('./data/preprocessed-meta-California.csv')

/tmp/ipykernel_2929/3049391663.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  rest_df = pd.read_csv('./data/preprocessed-meta-California.csv')


In [3]:
# Load stored data
%store -r rating_matrix_df
%store -r user_similarity_df

In [4]:
print(business_df)
print(user_df)
print(rest_df)

       index                               business
0          0  0x80c2c778e3b73d33:0xbdc58662a4a97d49
1          1  0x80dd2b4c8555edb7:0xfc33d65c4bdbef42
2          2  0x808f879f35b5088b:0xe3541cec7a95bd88
3          3  0x808f87f90c1f661f:0xf384e804a61e0c0b
4          4   0x80c2baf50d29bf63:0x5bd904b842b9fcc
...      ...                                    ...
80631  80631  0x80dd31128a6bd01d:0x3263b7a73387bd1a
80632  80632  0x80dd3124ea943e27:0xef87c1b9a4482e47
80633  80633  0x80dd3125a85f8ef3:0x13a546608196176c
80634  80634  0x80dd256e5bb86d89:0x12ec8ff3f2aedfa7
80635  80635  0x80dd4a7afea27289:0xe49cfab49567e5cb

[80636 rows x 2 columns]
           index                   user
0              0  107889750495339695586
1              1  108991152262655788985
2              2  111290322219796215751
3              3  112640357449611959087
4              4  117440349723823658676
...          ...                    ...
6323420  6323420  114477647681881742854
6323421  6323421  117313801155

In [5]:
print(rating_matrix_df)
print(user_similarity_df)

business  7185   7239   7330   7332   7341   7351   7377   7406   7421   \
user                                                                      
118           0      0      0      0      0      0      0      0      0   
128           0      0      0      0      0      0      0      0      0   
184           0      0      0      0      0      0      0      0      0   
199           0      4      0      0      0      0      0      0      0   
204           0      5      0      0      0      0      0      0      0   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
1244522       0      0      0      0      0      0      0      0      0   
1248454       0      0      0      0      0      0      0      0      0   
1311498       0      0      0      0      0      0      0      0      0   
1362023       0      0      0      0      0      0      0      0      0   
1997603       0      0      0      0      0      0      0      0      0   

business  7422   ...  80

In [6]:
def recommend_restaurants(user_id, num_recommendations):
    # 해당 사용자와 다른 사용자 간의 유사도를 구합니다.
    user_similarities = user_similarity_df.loc[user_id]
    
    # 해당 사용자의 레스토랑 평점을 구합니다.
    user_ratings = rating_matrix_df.loc[user_id][:1000]
    
    # 유사도 * 사용자의 레스토랑 평점을 계산합니다. 이 값이 높은 레스토랑일수록 추천됩니다.
    score = user_similarities.dot(rating_matrix_df)
    score = pd.Series(score, index=user_ratings.index)
    
    # 이미 평가한 레스토랑은 제외합니다.
    already_rated = user_ratings[user_ratings > 0].index
    score = score.drop(already_rated, errors='ignore')  # errors='ignore'를 추가하여 이미 평가한 레스토랑이 없을 경우 무시
    
    # 점수가 높은 순으로 정렬하고, 상위 n개의 레스토랑을 추천합니다.
    recommendations = score.nlargest(num_recommendations)
    
    return recommendations

In [7]:
# 사용자로부터 입력을 받아 사용자 ID 설정
while True:
    user_id_input = int(input("사용자 ID를 입력하세요: "))
    
    # 입력된 사용자 ID가 존재하는지 확인
    if user_id_input in rating_matrix_df.index:
        break
    else:
        print("해당 사용자 ID가 존재하지 않습니다. 다시 입력해주세요.")

사용자 ID를 입력하세요: 204


In [8]:
# 레스토랑 추천 함수 호출
recommend_df = recommend_restaurants(user_id_input, 10)
recommend_df.columns = ['business', 'score']

In [9]:
# Merge recommend_df with business_series based on the business column
result_df = pd.merge(recommend_df, business_df, how='left', left_on='business', right_index=True)

In [10]:
# 'user_id_input' 컬럼과 'index' 컬럼의 데이터 타입을 문자열로 변환
result_df[user_id_input] = result_df[user_id_input].astype(str)

# 'business' 컬럼을 인덱스로 설정
result_df.set_index('business', inplace=True)

# 인덱스를 열로 변환
result_df.reset_index(inplace=True)

# 'business' 컬럼의 데이터 타입을 문자열로 변환
result_df['business'] = result_df['business'].astype(str)

In [11]:
# Merge recommend_df and rest_df based on the common column
result_df_2 = pd.merge(result_df, rest_df, how='left', left_on='business', right_on='gmap_id')

In [12]:
result_df_2

,business,204,index,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,0x80858085de1e33eb:0x1a31405067f79d69,51.01865564703246,35785,Westfield San Francisco Centre,"Westfield San Francisco Centre, 865 Market St,...",0x80858085de1e33eb:0x1a31405067f79d69,9-story mall featuring over 170 shops & restau...,37.784118,-122.406386,"['Shopping mall', 'Clothing store', 'Departmen...",4.3,9998,NaN,"[['Thursday', '11AM–7PM'], ['Friday', '11AM–7P...",{'Accessibility': ['Wheelchair accessible elev...,Open ⋅ Closes 7PM,"['0x80858088e3ab84b5:0x190f3e428a962231', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,0x808580f112dc711d:0xbfb3fcde3487fb40,44.57320019077626,38162,Tony's Pizza Napoletana,"Tony's Pizza Napoletana, 1570 Stockton St, San...",0x808580f112dc711d:0xbfb3fcde3487fb40,Bustling Italian eatery with varied pizza opti...,37.800307,-122.409011,"['Pizza restaurant', 'Italian restaurant']",4.5,4480,$$,"[['Tuesday', '12–9PM'], ['Wednesday', '12–10PM...","{'Service options': ['Outdoor seating', 'No-co...",Open ⋅ Closes 9PM,"['0x808580f112dc711d:0xd142b7901abd9d7f', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,0x808580a24ec2f6f1:0x2117f92303bb4bf5,40.833379833453556,41946,Suppenküche,"Suppenküche, 525 Laguna St, San Francisco, CA ...",0x808580a24ec2f6f1:0x2117f92303bb4bf5,Authentic German fare like pretzels & schnitze...,37.776313,-122.426419,['German restaurant'],4.5,1381,$$,"[['Sunday', '10:30AM–2:30PM'], ['Monday', 'Clo...","{'Service options': ['No-contact delivery', 'D...",Closed ⋅ Opens 10:30AM,"['0x808580a26c77306d:0xb43108d1e99c8af1', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
3,0x808f7e28b17fa279:0xcaa79b9fbbc34ee,38.48772276545891,35824,SoMa StrEat Food Park,"SoMa StrEat Food Park, 428 11th St, San Franci...",0x808f7e28b17fa279:0xcaa79b9fbbc34ee,A rotating group of food trucks parked in an o...,37.769775,-122.412033,"['Food court', 'Barber shop', 'Restaurant']",4.3,1653,$$,"[['Thursday', '10AM–6PM'], ['Friday', '10AM–6P...","{'Service options': ['Takeout', 'Delivery'], '...",Open ⋅ Closes 6PM,"['0x808f7fcfddddc751:0xb5383aac2dbd7b2a', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
4,0x808f7e1806d86c9d:0xc9cb2a89fd8c5ede,38.089165732315266,29171,Pizzeria Delfina - Mission,"Pizzeria Delfina - Mission, 3611 18th St, San ...",0x808f7e1806d86c9d:0xc9cb2a89fd8c5ede,Relaxed urban pizza joint with some al fresco ...,37.761431,-122.424250,"['Restaurant', 'Pizza restaurant']",4.4,1178,$$,"[['Monday', '4:30–9PM'], ['Tuesday', '4:30–9PM...",{'From the business': ['Identifies as women-le...,Closed ⋅ Opens 4:30PM,"['0x808f7e2224c29931:0x234a4403d0bf9437', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
5,0x808f7b468cecc973:0x6ea332053be5ed5f,36.17417194799515,38082,Serramonte Center,"Serramonte Center, 3 Serramonte Center, Daly C...",0x808f7b468cecc973:0x6ea332053be5ed5f,"Clothing chain with trendy apparel, shoes & ac...",37.669597,-122.469405,"['Shopping mall', 'Grocery store', 'Restaurant']",4.2,6469,NaN,"[['Tuesday', '10AM–9PM'], ['Wednesday', '11AM–...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM Wed,"['0x808f7bf4bdca8f8d:0x7b6bc1e094dd114d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
6,0x808f7fd42a204f2b:0x62a37ae2b2392a2b,34.7300814045371,9342,Marlowe,"Marlowe, 500 Brannan St, San Francisco, CA 94107",0x808f7fd42a204f2b:0x62a37ae2b2392a2b,Stylish & lively option serving Californian co...,37.778342,-122.396789,"['American restaurant', 'Bar', 'Bistro', 'Brun...",4.4,1428,$$$,"[['Wednesday', '3:30–8:30PM'], ['Thursday', '3...","{'Service options': ['Takeout'], 'Highlights':...",Temporarily closed,"['0x808f7e310d3891a9:0xc9872041746a8751', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
7,0x808580668c955555:0x3f07d5248c09fe1f,32.838467524807776,29165,Gott's Roadside,"Gott's Roadside, One Ferry Building #6, San Fr...",0x808580668c955555:0x3f